<a href="https://colab.research.google.com/github/giridharinteg/AI_utils/blob/feature_bulk_delete/Bulk_delete_assistants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
import os
import pprint
from tqdm import tqdm

# Set your API key as an environment variable or directly in the script
API_KEY = input("Enter your OPEN AI API KEY HERE - ")
client = OpenAI(api_key = API_KEY)
response = client.beta.assistants.list(limit=20, order = 'desc')

def delete_assistant(assistant_id):
    response = client.beta.assistants.delete(assistant_id)
    return response

all_my_assistants = {}
to_delete_assistant_id = []

for assistant in response:
    #print(str(assistant.name) + " - " + str(assistant.id) + " - " + str(assistant.created_at))
    if assistant.name in all_my_assistants.keys():
        all_my_assistants[assistant.name]["duplicate_count"] = all_my_assistants[assistant.name]["duplicate_count"] + 1
        if assistant.created_at > all_my_assistants[assistant.name]["timestamp"]:
            to_delete_assistant_id.append(all_my_assistants[assistant.name]["id"])
            all_my_assistants[assistant.name]["id"] = assistant.id
            all_my_assistants[assistant.name]["timestamp"] = assistant.created_at
            all_my_assistants[assistant.name]["recent"] = True
            #print("REMOVAL OLD -  "+ str(assistant.created_at) + " > " + str(all_my_assistants[assistant.name]["timestamp"]))
        else:
            to_delete_assistant_id.append(assistant.id)
            #print("REMOVAL NEW -  "+ str(assistant.created_at) + " < " + str(all_my_assistants[assistant.name]["timestamp"]))
    else:
        all_my_assistants[assistant.name] = {}
        all_my_assistants[assistant.name]["id"] = assistant.id
        all_my_assistants[assistant.name]["timestamp"] = assistant.created_at
        all_my_assistants[assistant.name]["recent"] = True
        all_my_assistants[assistant.name]["duplicate_count"] = 0
        #print("ADDED")

print("Assistants Retained : ")
pprint.pprint(all_my_assistants)

print("Total Duplicate Assistants to be Deleted : "+str(len(to_delete_assistant_id)))

for assistant in tqdm(to_delete_assistant_id):
    resp = delete_assistant(assistant)
    if resp.deleted == True:
        pass
    else:
        print("Error happened but you can retry running without any issues. Assistant ID Below")
        print(resp.id)

print("All assistants have been deleted.")

In [ ]:
resp#deleted=True

In [ ]:
to_delete_assistant_id